In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
####################################
# 1. Veri Hazırlığı ve Augmentasyon
####################################
# Resimler 28x28 boyutunda; veri augmentasyonu ve normalize işlemleri uygulanıyor.
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Resimler zaten tek kanallı olabilir; emin olmak için kullanıldı.
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Klasör yapınız:
train_dataset = datasets.ImageFolder(root='train_images', transform=train_transforms)
test_dataset  = datasets.ImageFolder(root='test_images', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

num_classes = len(train_dataset.classes)
print("Toplam sınıf sayısı:", num_classes)

Toplam sınıf sayısı: 26


In [7]:
####################################
# 2. Basit ResNet Modeli Tanımı
####################################
# Residual blok tanımı
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

# Düzeltilmiş _make_layer fonksiyonu:
def _make_layer(block, in_channels, out_channels, blocks, stride):
    layers = []
    # İlk bloğun giriş kanalı belirtilen in_channels, çıkışı out_channels
    layers.append(block(in_channels, out_channels, stride))
    for i in range(1, blocks):
        layers.append(block(out_channels, out_channels))
    return nn.Sequential(*layers)

# Basitleştirilmiş ResNet mimarisi
class SimpleResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super(SimpleResNet, self).__init__()
        # İlk conv katmanı: 28x28 görüntüler için uygun ayar (kanal sayısı: 1 -> 16)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        # Layer1: giriş kanalı 16, çıkış kanalı 16
        self.layer1 = _make_layer(block, in_channels=16, out_channels=16, blocks=layers[0], stride=1)
        # Layer2: giriş kanalı 16, çıkış kanalı 32 (stride ile downsampling)
        self.layer2 = _make_layer(block, in_channels=16, out_channels=32, blocks=layers[1], stride=2)
        # Layer3: giriş kanalı 32, çıkış kanalı 64 (stride ile downsampling)
        self.layer3 = _make_layer(block, in_channels=32, out_channels=64, blocks=layers[2], stride=2)
        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))  # [B, 16, 28, 28]
        x = self.layer1(x)                      # [B, 16, 28, 28]
        x = self.layer2(x)                      # [B, 32, 14, 14]
        x = self.layer3(x)                      # [B, 64, 7, 7] (örnek boyutlar, padding/stride ayarlarına bağlı)
        x = self.avgpool(x)                     # [B, 64, 1, 1]
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [8]:
# Örneğin, her residual katmanda 1 blok kullanılarak model oluşturuluyor.
resnet = SimpleResNet(BasicBlock, [1, 1, 1], num_classes=num_classes)
print(resnet)

SimpleResNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride

In [10]:
####################################
# 3. Model Eğitim Döngüsü ve Kaydetme
####################################
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=0.001, weight_decay=1e-4)  # weight_decay L2 regularizasyon için

num_epochs = 20
best_test_loss = np.inf
patience = 5
counter = 0  # erken durdurma için

for epoch in range(num_epochs):
    resnet.train()
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images, labels
        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
    train_loss /= len(train_loader.dataset)
    
    # Test/Doğrulama adımı
    resnet.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images, labels
            outputs = resnet(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
    test_loss /= len(test_loader.dataset)
    accuracy = correct / len(test_loader.dataset)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    
    # Erken durdurma: Eğer test loss iyileşmezse art arda patience kadar epoch sonrasında eğitim durdurulur.
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        counter = 0
        # En iyi modelin ağırlıklarını kaydet
        torch.save(resnet.state_dict(), "best_resnet_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("Erken durdurma tetiklendi!")
            break

print("ResNet modeli eğitildi ve en iyi ağırlıklar 'best_resnet_model.pth' olarak kaydedildi.")

Epoch [1/20], Train Loss: 0.6189, Test Loss: 0.2805, Test Accuracy: 0.9195
Epoch [2/20], Train Loss: 0.5045, Test Loss: 0.2464, Test Accuracy: 0.9277
Epoch [3/20], Train Loss: 0.4410, Test Loss: 0.2414, Test Accuracy: 0.9290
Epoch [4/20], Train Loss: 0.4040, Test Loss: 0.2141, Test Accuracy: 0.9370
Epoch [5/20], Train Loss: 0.3749, Test Loss: 0.1981, Test Accuracy: 0.9408
Epoch [6/20], Train Loss: 0.3540, Test Loss: 0.1906, Test Accuracy: 0.9450
Epoch [7/20], Train Loss: 0.3417, Test Loss: 0.1688, Test Accuracy: 0.9511
Epoch [8/20], Train Loss: 0.3301, Test Loss: 0.1682, Test Accuracy: 0.9510
Epoch [9/20], Train Loss: 0.3197, Test Loss: 0.1758, Test Accuracy: 0.9479
Epoch [10/20], Train Loss: 0.3100, Test Loss: 0.1686, Test Accuracy: 0.9509
Epoch [11/20], Train Loss: 0.3047, Test Loss: 0.1534, Test Accuracy: 0.9543
Epoch [12/20], Train Loss: 0.2983, Test Loss: 0.1542, Test Accuracy: 0.9564
Epoch [13/20], Train Loss: 0.2962, Test Loss: 0.1521, Test Accuracy: 0.9569
Epoch [14/20], Train 

In [11]:
####################################
# 4. Confusion Matrix Hesaplama ve Kaydetme
####################################
# Test verileri üzerinde modelin tahminlerini toplayıp confusion matrix oluşturuluyor.
all_preds = []
all_labels = []

resnet.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images
        outputs = resnet(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())

# Confusion matrix hesaplanıyor
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Confusion matrix'in görselleştirilmesi
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")
plt.close()
print("Confusion matrix 'confusion_matrix.png' olarak kaydedildi.")


Confusion Matrix:
 [[1169    0    0    1    0    3    0   14    0    0    2    0    2    2
     1    1    2    2    0    0    0    1    0    0    0    0]
 [   3 1145    1    9    7    0    4    0    0    0    0    0    0    0
     9    2    4   14    2    0    0    0    0    0    0    0]
 [   0    0 1183    0    3    0    1    0    1    0    0    7    0    0
     1    0    0    0    0    0    2    0    0    0    1    1]
 [   9    3    1 1088    0    1    0    1    0    3    0    0    3    2
    76    3    6    0    1    0    0    1    0    0    2    0]
 [   0    0    6    0 1169   16    5    0    1    0    0    0    1    0
     0    0    0    0    0    0    0    0    0    0    0    2]
 [   0    0    2    0    7 1171    2    0    0    2    0    2    0    0
     0    4    2    2    2    2    0    0    0    0    2    0]
 [   1    8   10    2    5    1 1148    1    0    0    0    0    0    0
     2    5   11    2    2    0    2    0    0    0    0    0]
 [   1    0    0    0    0    2    0